In [66]:
import xml.etree.ElementTree as ET
import pandas as pd

In [67]:
def extract_data_brut(file_path,output_csv_path):
    '''
    extraire les textes et leurs parities à partir des corpus d'apprentissage bruts
    et les conserver dans un fichier csv pour chaque langue
    '''    
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for doc in root.findall('.//doc'):
        # obtenir tous les paragraphes du texte et les joindre en un seul
        text_elements = doc.findall('.//texte//p')
        texte = ' '.join([elem.text.strip() for elem in text_elements if elem.text])
        partis = [parti.get('valeur') for parti in doc.findall('.//PARTI')]
        for parti in partis:
            data.append({'texte': texte, 'parti': parti})

    df = pd.DataFrame(data)
    return df.to_csv(output_csv_path, index=False, encoding='utf-8')

In [68]:
def extract_and_combine_text_label(xml_file_path, label_file_path, output_csv_path):
    '''
    extraire les textes tests dans les fichiers xml de et leurs labels des fichiers textes 
    et les combiner dans un seul fichier csv pour chaque langue
    '''   
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    test_data = []
    for doc in root.findall('.//doc'):
        # obtenir tous les paragraphes du texte et les joindre en un seul
        text_elements = doc.findall('.//texte//p')
        texte = ' '.join([elem.text.strip() for elem in text_elements if elem.text])
        test_data.append({'texte': texte})

    texte_df = pd.DataFrame(test_data)

    # lecture des labels à partir du fichier texte et les combiner avec les textes dans un même dataframe
    labels_df = pd.read_csv(label_file_path, sep='\t', header=None, names=['id', 'label'])
    labels_df.set_index('id', inplace=True)

    # ajouter une colonne id pour pouvoir joindre les deux dataframes
    texte_df['id'] = range(1, len(texte_df) + 1)
    texte_df.set_index('id', inplace=True)

    # joindre les deux dataframes en utilisant l'index id comme clé de jointure 
    combined_df = texte_df.join(labels_df, how='left')
    return combined_df.to_csv(output_csv_path, index=False)

In [27]:
extract_data_brut("corpus/deft09_parlement_appr_xml/deft09_parlement_appr_en.xml","corpus/output_en_app.csv")

In [28]:
# pretraitement

In [29]:
import nltk
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# installer les packages nltk nécessaires pour la préparation des données 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xiaohua/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/xiaohua/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
def preprocess_text(text, lang):
    '''
    prétraiter le texte en appliquant les étapes suivantes:
    - convertir le texte en minuscules
    - supprimer les nombres
    - supprimer les apostrophes
    - supprimer les mots de longueur inférieure ou égale à 2
    - supprimer les stopwords et la ponctuation
    '''
    # assurer que le texte est de type str
    if not isinstance(text, str):
        return str(text)  # convertir en str

    # charger les stopwords selon la langue
    if lang == 'en':
        stop_words = set(stopwords.words('english'))
    elif lang == 'fr':
        stop_words = set(stopwords.words('french'))
    elif lang == 'it':
        stop_words = set(stopwords.words('italian'))

    # enlever les symboles ennuyeux et les nombres 
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r"['’‘]", ' ', text)  
    text = re.sub(r'\b\w{1,2}\b', '', text)  

    words = word_tokenize(text.lower())

    # enlever les stopwords et la ponctuation
    words = [word for word in words if word not in stop_words and word not in string.punctuation]

    cleaned_text = ' '.join(words)
    return cleaned_text

In [50]:
def preprocess_csv(csv_path, lang,output_csv_path):
    '''
    lecture du fichier csv contenant les textes et leurs labels
    et prétraitement des textes
    '''
    # lire le fichier csv sauvegardé précédemment
    df = pd.read_csv(csv_path)

    # appliquer la fonction de prétraitement sur la colonne texte
    df['texte'] = df['texte'].apply(lambda x: preprocess_text(x, lang))

    df.to_csv(output_csv_path, index=False, encoding='utf-8')

In [51]:
preprocess_csv("/Users/xiaohua/Desktop/Cours/M2_Paris/Apprentissage_auto/proj_futo/corpus/output_en_app.csv","en","/Users/xiaohua/Desktop/Cours/M2_Paris/Apprentissage_auto/proj_futo/corpus/output_en_app_traite.csv")

In [52]:
#!/usr/bin/env python3

from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm


In [55]:
def tfidfVectorize(CSV_train, CSV_test):
    '''
    vectoriser les textes en utilisant la méthode TF-IDF
    '''
    train_df = pd.read_csv(CSV_train)
    test_df = pd.read_csv(CSV_test)
    train_df.dropna(subset=['texte'], inplace=True)
    test_df.dropna(subset=['texte'], inplace=True)


    # initialiser le vectorizer avec les paramètres désirés
    vectorizer = TfidfVectorizer(
        ngram_range=(1, 3),
        max_df=0.5,
        use_idf=True,
        sublinear_tf=True,
        max_features=10000
    )

    # vectoriser les textes d'entraînement et de test 
    print("Vectorizing train data...")
    X_train = vectorizer.fit_transform(tqdm(train_df['texte']))

    print("Vectorizing test data...")
    X_test = vectorizer.transform(tqdm(test_df['texte']))

    # récupérer les labels correspondants
    y_train = train_df['parti'].astype(str)
    y_test = test_df['parti'].astype(str)

    return X_train, X_test, y_train, y_test


In [56]:
tfidfVectorize("/Users/xiaohua/Desktop/Cours/M2_Paris/Apprentissage_auto/5eme_DEFT_Fouille_Opinion/data/texte_cleaned/train_text_cleaned_en.csv","/Users/xiaohua/Desktop/Cours/M2_Paris/Apprentissage_auto/5eme_DEFT_Fouille_Opinion/data/texte_cleaned/test_text_cleaned_en.csv")

Vectorizing train data...



100%|███████████████████████████████████| 19365/19365 [00:04<00:00, 4362.90it/s]


Vectorizing test data...



100%|███████████████████████████████████| 12911/12911 [00:02<00:00, 5586.45it/s]


(<19365x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 2517771 stored elements in Compressed Sparse Row format>,
 <12911x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 1662430 stored elements in Compressed Sparse Row format>,
 0          GUE-NGL
 1           PPE-DE
 2           PPE-DE
 3           PPE-DE
 4              PSE
            ...    
 19361         ELDR
 19362       PPE-DE
 19363    Verts-ALE
 19364         ELDR
 19365         ELDR
 Name: parti, Length: 19365, dtype: object,
 0              PSE
 1           PPE-DE
 2        Verts-ALE
 3           PPE-DE
 4             ELDR
            ...    
 12908       PPE-DE
 12909         ELDR
 12910          PSE
 12911          PSE
 12912    Verts-ALE
 Name: parti, Length: 12911, dtype: object)

In [59]:
#!/usr/bin/env python3
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [60]:
def evaluate(X_train, X_test, y_train, y_test, model):
    '''
    Évaluer les performances du modèle choisi
    en calculant l'accuracy, le kappa et la matrice de confusion
    '''
    # Entraîner le modèle sur les données d'entraînement
    model.fit(X_train, y_train)

    # Prédire les labels sur les données de test
    y_pred = model.predict(X_test)

    return y_test, y_pred

In [ ]:
evaluate()

In [61]:
def print_scores(y_test, y_pred):
    '''
    Afficher les scores d'évaluation du modèle choisi
    '''
    accuracy = accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print("Accuracy:", accuracy)
    print("Kappa Score:", kappa)
    print("Classification Report:\n", report)

In [62]:
def plot_confusion_matrix(y_test,y_pred, model):
    '''
    Sauvagarder la visualisation de la matrice de confusion du modèle choisi
    '''
    # obtenir les labels prédits par le modèle
    unique_labels = sorted(set(y_test) | set(y_pred))

    # générer la matrice de confusion en spécifiant les labels
    conf_matrix = confusion_matrix(y_test, y_pred, labels=unique_labels)

    # utiliser seaborn pour afficher la matrice de confusion sous forme de heatmap
    plt.figure(figsize=(6, 4.5))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greys', 
                xticklabels=unique_labels, yticklabels=unique_labels)

    plt.xlabel('Parti Prédit')
    plt.ylabel('Parti Réel')
    plt.title('Matrice de Confusion')

    # sauvegarder la figure dans le dossier result
    plt.savefig(f'../result/Confusion_Matrix_{model}.png', format='png', dpi=300)
    
    plt.show()